# Finding historical PDUFA dates:
After getting in touch with the maintainers of [FDA Tracker](https://www.fdatracker.com/fda-calendar/), I had a better understanding of the resources at hand. Unfortunately, the FDA does not publish PDUFA dates. Some groups do however, and I was able to use thier google calendar to streamline the process. 

After validating the model, I can hopefully justify the time commitment to scrape 278 individual pharmaceutial companies websites, or begin keeping a running database of PDUFA dates scraped in my other notebook, `scrapingFuturePdufas.ipynb`.

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect
from datetime import datetime
from tqdm import tqdm_notebook
import dill

In [2]:
engine = create_engine('sqlite:///training_data.db')

First things first, lets import the needed librararies for working with the DB

...and one for working with the [.ics formatted list of historical FDA dates](https://calendar.google.com/calendar/ical/5dso8589486irtj53sdkr4h6ek%40group.calendar.google.com/public/basic.ics) which I [pulled down from google calendar](https://support.google.com/calendar/answer/37111?hl=en). 

In [3]:
from urllib2 import urlopen
import ics

...Lets open up that calendar

In [4]:
FdaUrl = "https://calendar.google.com/calendar/ical/5dso8589486irtj53sdkr4h6ek%40group.calendar.google.com/public/basic.ics"

In [5]:
FdaCal = ics.Calendar(urlopen(FdaUrl).read().decode('iso-8859-1'))

In [6]:
FdaCal

<Calendar with 543 events>

So I'm gonna need to get the stock ticker symbols out of each event name, and the start/end dates, which appear mostly to be all day one day events. Sounds like a job for...
###### REGULAR EXPRESSIONS DUM-DA-DUM

In [7]:
import re

In [8]:
tickerRe = re.compile(r"\A[A-Z]{3,4}\W")

In [9]:
past_pdufa = []
for event in FdaCal.events:
    matches = re.findall(tickerRe, event.name)
    if len(matches) >=1:
        eDate = event.begin.datetime
        '''eYr = event.begin.datetime.year
        eMn = event.begin.datetime.month
        eDy = event.begin.datetime.day'''
        eComp = str(matches[0]).strip()
        past_pdufa.append((eComp, eDate))

So we have a list of tuples containing a stock ticker and an FDA action date

I'm going to grab the symbols and feed those into a pickle, so we can pass them back to the API downloading notebook.

In [10]:
past_ticks = set(x[0] for x in past_pdufa)

In [11]:
dill.dump(past_ticks, open('past_ticker_symbols.pkl', 'w'))

So here is our list of **(475)** stock tickers in the past PDUFA dataset, including a few strings that slipped past the regex like `NEW` and `INC`. I'm going to run this list of ticker symbols against AlphaVantage to finish rounding out my training dataset. That _should_ clean them out and I can begin feature extraction tommorow.

Now lets go ahead and convert the ugly list of tuples to a nice pandas DF and write it to our SQLite DB

In [12]:
hist_pdufas = pd.DataFrame(past_pdufa, columns = ['Ticker', 'DateTime'])

In [13]:
hist_pdufas

,Ticker,DateTime
0,NEW,2006-10-06 00:00:00+00:00
1,DOR,2007-07-21 00:00:00+00:00
2,SPPI,2007-08-15 00:00:00+00:00
3,NBIX,2007-08-21 00:00:00+00:00
4,ZGEN,2007-10-18 00:00:00+00:00
5,DOR,2007-10-21 00:00:00+00:00
6,REGN,2007-11-29 00:00:00+00:00
7,NBIX,2007-12-12 00:00:00+00:00
8,JAZZ,2007-12-22 00:00:00+00:00
9,PCYC,2007-12-31 00:00:00+00:00


In [14]:
#hist_pdufas.set_index(['DateTime'], inplace = True)

SQLite cant accept date formatting so I'm gonna see how the stock series are stored by date and just copy thier style

In [15]:
hist_pdufas.to_sql("historical_pdufas", engine, if_exists = 'replace')

ValueError: Cannot cast DatetimeIndex to dtype datetime64[us]